# 협업필터링(CF) 추천 - Significance weighting

In [1]:
import numpy as np
print(f'NumPy v{np.__version__}')

import pandas as pd
print(f'pandas v{pd.__version__}')

import sklearn
print(f'scikit-learn v{sklearn.__version__}')

# Only for specifying versions
import sys; print(f'Python v{sys.version}')

NumPy v1.25.0
pandas v1.5.3
scikit-learn v1.2.0
Python v3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]


## 3.7 그 외의 CF 정확도 개선 방법

CF의 정확성을 더 개선할 수 있는 방법으로는 신뢰도 가중(significance weighting)이 있다.

예를 들어서 사용자 A에 대한 추천을 하는 경우를 생각해보자. 어떤 사용자는 사용자 A와 공통으로 평가한 아이템이 10개인 경우도 있고 또 다른 사용자는  사용자 A와 공통으로 평가한 아이템이 2개밖에 없는 경우가 있을 수 있다. 이 경우 사용자 A와 이 두 사용자 간의 유사도가 비슷하더라도 유사도의 신뢰도는 당연히 공통 아이템이 10개인 경우가 높다고 할 수 있다.

그래서 등장한 아이디어가 *유사도를 신뢰도에 따라서 가중을 하자*는 것이다. 공통 아이템이 많은 사용자와의 유사도에 공통 아이템이 적은 사용자와의 유사도보다 더 큰 가중치를 주자는 것이다. 그런데 예측값은 매우 민감해서 계산식을 약간만 바꿔도 RMSE 값이 크게 변동하기 때문에 공통 아이템의 수를 가중치로 직접 사용하면 얻는 것보다 잃는 것이 클 가능성이 높다.

한 가지 방법은 *신뢰도가 일정 이상인 사용자만을 예측치 계산에 사용*하는 것이다. 즉 *신뢰도(공통으로 평가한 아이템의 수)가 일정 이상인 사용자만 이웃 사용자로 활용*하는 것이다.

비슷한 원리로 예측값을 계산하는 대상 아이템에 대해서도 *평가한 사용자가 일정 수준 이상이 되는 아이템만 예측치를 계산*하도록 하는 것도 가능하다. 앞의 `3-2.ipynb`와 `3-3.ipynb`에서도 해당 영화를 평가한 사용자가 2명 이상인 경우만 계산을 하도록 했던 것을 기억할 것이다. 이것을 일반화하면 된다.

아래의 cell은 `3-3.ipynb`의 2~5번째 cell과 동일한 코드이다.

In [2]:
# 데이터 읽어 오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# timestamp 제거 
ratings.drop('timestamp', axis='columns', inplace=True)

# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)

# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis='columns')
rating_bias = (rating_matrix.T - rating_mean).T

In [3]:
# 사용자별 공통 평가 수 계산
rating_binary = (rating_matrix > 0).astype(float)
counts = rating_binary @ rating_binary.T    # Matrix product @

def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]

        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]

        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]

        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL

        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings.drop(none_rating_idx, inplace=True)
        sim_scores.drop(none_rating_idx, inplace=True)

        if neighbor_size == 0:      # Neighbor size가 지정되지 않은 경우
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction += rating_mean[user_id]

        else:                       # Neighbor size가 지정된 경우
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))

                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = sim_scores.to_numpy()
                movie_ratings = movie_ratings.to_numpy()

                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                selected_idx = user_idx[-neighbor_size:]

                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[selected_idx]
                movie_ratings = movie_ratings[selected_idx]

                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction += rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]

    # if prediction < 1:
    #     return 1
    # if prediction > 5:
    #     return 5

    return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
score(CF_knn_bias_sig, 30)

0.9478705279321811

RMSE가 0.948로, 신뢰도와 최소 사용자 수를 고려하지 않은 `3-3.ipynb`의 `CF_knn_bias()`보다 아주 조금 향상되었음을 볼 수 있다.

## 연습문제

Q. 코드를 수정하여 예측값이 1 이하이면 1로, 5 이상이면 5로 수정하는 코드를 작성해서 실행해보고 위의 결과와 비교하라.

위의 line 59~62의 주석처리된 부분을 지우고 실행해보면, RMSE가 0.947로 clipping 후가 아주 조금 향상되었음을 볼 수 있다.